In [1]:
import openai
import pandas as pd
import json
from langchain.agents import tool

In [2]:
input_data = pd.read_excel('RPA Pricing Extraction.xlsx')

In [3]:
output_data = input_data.copy()

In [4]:
output_data["Max Discount"]=None
output_data["IBD Bank Name/UPI"]=None
output_data["Min Swipe"]=None
output_data["Bump up"]=None
output_data["VPC(Amazon)"]=None
output_data["SuperCoins(Flipkart)"]=None
output_data.drop('BUP', axis=1, inplace=True)
output_data.drop('VPC', axis=1, inplace=True)
output_data.drop('SuperCoins', axis=1, inplace=True)
output_data

,Market Place,FSN /ASIN,Description,MRP,Current Price,Bank Offers,URL,Max Discount,IBD Bank Name/UPI,Min Swipe,Bump up,VPC(Amazon),SuperCoins(Flipkart)
0,Amazon,B0BKKHFGCP,"Lava Blaze 5G (Glass Blue, 6GB RAM, UFS 2.2 12...",16349,11999,1. 7.5% Instant Discount up to INR 1000 on Cit...,https://www.amazon.in/dp/B0BSTTK7HS?th=1,None,None,None,None,None,None
1,Flipkart,MOBGHDXFPWKFSFJ2,"REDMI Note 11 Pro (Stealth Black, 128 GB) (8 ...",24999,23999,"1. 10% off on Federal Bank Credit Card Txns, u...",https://www.flipkart.com/redmi-note-11-pro-ste...,None,None,None,None,None,None


In [5]:
def create_content(price, offers):
    content=f"The Price of the SKU is {price}\n\nOffers:\n\n{offers}\n\nWhich is the best Offer?"
    return content

In [54]:
def process_discount_details(max_discount, min_swipe):
    discount_perc = max_discount/min_swipe*100
    return discount_perc

In [55]:
#Load the LLM
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model='gpt-4-0613', temperature=0)

In [56]:
tools = []

In [58]:

@tool
def best_offer_details(max_discount, bank_name, min_swipe):
    '''Get the discount details like discount amount, bank name and Minimum swipe/pucrhase from the offer which is resulting in maximum discount
    
    Inputs:
        max_discount (integer): The discount amount from the best offer
        bank_name (str): The name of the bank which is offereing the best offer
        min_swipe (int): The minimum swipe or purchase amount from the best offer
    '''
    print("Discount%", process_discount_details(max_discount, min_swipe))
tools = [best_offer_details]

In [59]:
#Create the prompt

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant which helps in the identification of the best offer. In any offer when not explicitly mentioned assume any percentage discount/cashback as percentage of SKU price. Assume membership if that is one of the condition for any offer. Assume Prime membership wherever applicable & transactions are not Amazon business transactions."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [60]:
#Bind the tool with the LLM
from langchain.tools.render import format_tool_to_openai_function
llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

In [61]:
#Create the Agent

from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

In [62]:
#Create the agent_executor

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [63]:
response = agent_executor.invoke({"input": "The Price of the SKU is 15999\n\nOffers:\n\n1. Flat INR 2100 Instant Discount on HDFC Bank Credit CardTxn. Minimum purchase value INR 17999\n\n2. Flat INR 2100 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit CardTxn. Minimum purchase value INR 17999\n\n3. Flat INR 2000 Instant Discount on HDFC Bank Debit Card EMI Txn. Minimum purchase value INR 19999\n\n4. 5% Instant Discount up to INR 250 on HSBC Cashback Card Credit Card Transactions. Minimum purchase value INR 1000\n\n5. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n\n6. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later.\n\nWhich is the best Offer?"})



> Entering new AgentExecutor chain...

Invoking: `best_offer_details` with `{'max_discount': 250, 'bank_name': 'HSBC', 'min_swipe': 1000}`
responded: The best offer would be "5% Instant Discount up to INR 250 on HSBC Cashback Card Credit Card Transactions. Minimum purchase value INR 1000" as it is the only offer that is applicable for the given SKU price of INR 15999. All other offers require a minimum purchase value that is higher than the SKU price.

Now, let's calculate the discount:

5% of 15999 = 799.95. But the maximum discount is capped at INR 250. So, the discount that can be availed is INR 250.

Let's use the best_offer_details function to get the details of the best offer.

Discount% 25.0
NoneThe best offer is a 5% Instant Discount up to INR 250 on HSBC Cashback Card Credit Card Transactions with a minimum purchase value of INR 1000.

> Finished chain.


In [53]:
response['intermediate_steps'][0][0].tool_input

{'max_discount': 799.95,
 'bank_name': 'Amazon Pay ICICI Bank',
 'min_swipe': 15999}

In [24]:
response

{'input': 'The Price of the SKU is 15999\n\nOffers:\n\n1. Flat INR 2100 Instant Discount on HDFC Bank Credit CardTxn. Minimum purchase value INR 17999\n\n2. Flat INR 2100 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit CardTxn. Minimum purchase value INR 17999\n\n3. Flat INR 2000 Instant Discount on HDFC Bank Debit Card EMI Txn. Minimum purchase value INR 19999\n\n4. 5% Instant Discount up to INR 250 on HSBC Cashback Card Credit Card Transactions. Minimum purchase value INR 1000\n\n5. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n\n6. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later.\n\nWhich is the best Offer?',
 'output': 'The best offer is a 5% Instant Discount up to INR 800 on HSBC Cashback Card Credit Card Transactions with a minimum purchase value of INR 1000.',
 'intermediate_steps': [(AgentActionMessageLog(tool='offer_formatted_output'